1. Напишите декоратор, реализующий проверку по заданному правилу входных аргументов
функции ( @pre ) .При невыполнении условий прерывайте выполнение с выдачей заранее
заданного сообщения. Декоратор должен принимать в качестве аргументов правило и
сообщение

In [1]:
def pre(param):
    def wrapper(func):
        def inner(*args, **kwargs):
            [values] = args + tuple(sorted(kwargs.values()))
            for i in values:
                assert param(i), 'invalid argument'
            return func(*args, **kwargs)
        return inner
    return wrapper


In [2]:
@pre(lambda l: l < 10)
def print_array(array):
    print(array)

In [3]:
array_1 = [1, 2, 3, 4, 9]
array_2 = [1, 22, 3, 4, 9]
array_3 = [100, 22, 103, 104, 109]

In [4]:
print_array(array_1)
print_array(array_2)

[1, 2, 3, 4, 9]


AssertionError: ignored

2. Напишите декоратор, реализующий проверку по заданному правилу выходных аргументов
функции ( @post ) . Все остальное - как в примере выше.

In [5]:
def post(param):
    def wrapper(func):
        def inner(*args, **kwargs):
            values = func(*args, **kwargs)
            for i in values:
                assert param(i), 'invalid argument'
            return values
        return inner
    return wrapper


In [6]:
@post(lambda l: l>10)
def return_array(array):
    return array

In [7]:
print(return_array(array_3))
return_array(array_1)

[100, 22, 103, 104, 109]


AssertionError: ignored

3. Напишите настраиваимый декоратор, обеспечивающий мемоизацию (проверьте его на
функции Аккермана или на вычислении n-го числа Фибоначчи). Мемоизатор должен хранить
не более n значений, (как в очереди -- если пришло n+1 -ое) -- вы забываете 1 -ое. После
выполнения функции выдавайте на печать сколько раз функция посчиталась, и сколько раз вы
взяли значение из памяти

In [105]:
def memo(param):
    def wrapper(func):
        cache = dict()
        counter = 0
        count_cache = 0

        def inner(*args, **kwargs):
            nonlocal counter
            nonlocal count_cache
            values = args + tuple(sorted(kwargs.items()))

            if values in cache:
                output = cache[values]
                count_cache += 1
            else:
                output = func(*args, **kwargs)
                counter += 1
                if param(len(cache)):
                    cache[values] = output
                else:
                    cache[values] = cache.pop(list(cache)[0])
                    cache[values] = output
            return output, counter, count_cache
        return inner
    return wrapper

In [106]:
@memo(lambda l: l <= 20)
def fib(num):
    if num == 1 or num == 2:
        return 1
    return fib(num-1) + fib(num-2)

In [107]:
a, b, c = fib(200)
print(b, c)


200 197


4. Напишите декоратор определяющий среднее выполнение времени функции по результатам n
запусков


In [19]:
import time

In [100]:
def mid_time(n=10):
  def wrapper(func):
    list_mid_time = list()
    def inner(*args, **kwargs):
      for i in range(n):
        tick = time.perf_counter()
        result = func(*args, **kwargs)
        list_mid_time.append(time.perf_counter() - tick)

      return sum(list_mid_time)/(len(list_mid_time))

    return inner
  return wrapper


In [101]:
def check_func(func):
  cache = dict()
  def inner(*args, **kwargs):
    values = args + tuple(sorted(kwargs.items()))
    if values not in cache:
      cache[values] = func(*args, **kwargs)
    return cache[values]
  return inner

In [102]:
@check_func
@mid_time(100)
def fib(num):
    if num == 1 or num == 2:
        return 1
    return fib(num-1) + fib(num-2)

In [103]:
fib(3)

1.9613999954041597e-06

In [104]:
fib(30)

5.1598330667729895e-05